In [ ]:
from nibabel import trackvis as tv
from dipy.tracking.streamline import Streamlines
from dipy.data import get_data
from dipy.io.streamline import load_trk

# Tractography Clustering

In [ ]:
import viztools
brayns = viztools.Brayns('localhost:5000')

In [ ]:
def add_streamlines(streams, radius):
    ''' Build streamlines'''
    count = 0
    indices = []
    points = []
    for stream in streams:
        for x in stream:
            if x is not None:
                indices.append(count)
                for y in x:
                    for z in y:
                        points.append(float(z))
                count = count + len(x)
        
    ''' RPC call to Brayns '''
    params = dict()
    params['name'] = 'fornix'
    params['indices'] = indices
    params['points'] = points
    params['radius'] = radius
    params['opacity'] = 1
    result = brayns.rpc_request("streamline", params=params)

In [ ]:
fname = get_data('fornix')
streams, hdr = tv.read(fname)

add_streamlines(streams, 0.1)

## http://tractome.org

In [ ]:
def add_streamlines2(name, streams, radius, opacity, range = (0, 1e6)):
    ''' Build streamlines'''
    count = 0
    indices = []
    points = []
    for stream in streams[range[0]:range[1]]:
        indices.append(count)
        for x in stream:
            for z in x:
                    points.append(float(z))
        count = count + len(stream)
        
    ''' RPC call to Brayns '''
    params = dict()
    params['name'] = name
    params['indices'] = indices
    params['points'] = points
    params['radius'] = radius
    params['opacity'] = opacity
    result = brayns.rpc_request("streamline", params=params)

In [ ]:
streams, hdr = load_trk('/home/favreau/medias/dti/HCP_subject124422_100Kseeds/tracks_dti_100K.trk')
print(len(streams))
streams_per_model = 1000
for i in range(1, len(streams), streams_per_model):
    add_streamlines2(str(i) + '-' + str(i + streams_per_model - 1), streams, 0.1, 1, (i, i + streams_per_model - 1))

In [ ]:
streams, hdr = load_trk('/home/favreau/medias/dti/HCP_subject124422_3Mseeds/tracks_dti_3M.trk')
print(len(streams))
streams_per_model = 1000
for i in range(1, len(streams), streams_per_model):
    add_streamlines2(str(i) + '-' + str(i + streams_per_model - 1), streams, 0.1, 1, (i, i + streams_per_model - 1))